In [1]:
from pymongo import MongoClient
import numpy as np
import json
import gensim
client = MongoClient("mongodb://localhost:27017")
db = client['chpreprcs_db']
coll = db['chpreprcs_coll']
print(gensim.__version__)
print(np.__version__)
print(json.__version__)

0.13.1
1.11.1
2.0.9


In [2]:
k = 12938  # k = 1 ~ 1395958
_COLL_NUM_ = 1395958
_VEC_SIZE_ = 100
_SYN_VEC_SIZE_ = 30
doc = coll.find_one({"_id": k})['sentence']
print(doc)
index = 4
a = model[doc[index + 1]]
b = model[doc[index - 1]]
c = model[doc[index + 2]]
d = model[doc[index - 2]]
print('\n')
print(model.most_similar([a + b + c + d]))

['我', '不禁', '對', '山西', '商人', '深深', '地', '敬佩', '起來', '。']


NameError: name 'model' is not defined

In [6]:
class MySentences(object):
    def __init__(self, collection):
        self.coll = collection
    def __iter__(self):
        for k in range(1, _COLL_NUM_ + 1):
            doc = self.coll.find_one({"_id": k})
            s = doc['sentence']
            if len(s) > 5:
                yield s
class MySyntax(object):
    def __init__(self, collection):
        self.coll = collection
    def __iter__(self):
        for k in range(1, _COLL_NUM_ + 1):
            doc = self.coll.find_one({"_id": k})
            s = doc['syntax_label']
            if len(s) > 5:
                yield s
sentences = MySentences(coll)
syntax = MySyntax(coll)

In [ ]:
# model = gensim.models.Word2Vec(sentences, window = 5, min_count = 10, size = _VEC_SIZE_, iter = 30)
# model.save('w2v_chtag_35')
# model_syntax = gensim.models.Word2Vec(syntax, window = 3, min_count = 10, size = _SYN_VEC_SIZE_, iter = 30)
# model_syntax.save('w2v_chtag_syntax_35')

<bound method Word2Vec.train of <gensim.models.word2vec.Word2Vec object at 0x7f5f895449e8>>


In [4]:
print(model.most_similar(['張藝謀']))
print(model_syntax.most_similar(['PERIODCATEGORY']))

[('楊德昌', 0.6508117914199829), ('張作驥', 0.5904040932655334), ('瑪丹娜', 0.5871766209602356), ('音樂劇', 0.5689635276794434), ('阿呆', 0.568121612071991), ('王家衛', 0.5637759566307068), ('張毅', 0.5585370659828186), ('電視劇', 0.5552148818969727), ('鄭秀文', 0.5549250841140747), ('港劇', 0.5493459105491638)]
[('COMMACATEGORY', 0.8969951272010803), ('SEMICOLONCATEGORY', 0.828833818435669), ('D', 0.7599872946739197), ('Cbb', 0.6997944116592407), ('Na', 0.6980289816856384), ('Neqa', 0.6058214902877808), ('VH', 0.5926657915115356), ('Caa', 0.5728498101234436), ('A', 0.5615950226783752), ('QUESTIONCATEGORY', 0.5482258796691895)]


In [3]:
model = gensim.models.Word2Vec.load('w2v_chtag_35')
model_syntax = gensim.models.Word2Vec.load('w2v_chtag_syntax_35')

In [249]:
import random
import numpy as np

def make_cloze(num):
    if num > 1000: num = 1000
        
    idx = []
    for i in range(num):
        idx.append(round(random.uniform(1, _COLL_NUM_)))
    idx = sorted(set(idx[: num]))
    
    sentences = []
    for i in idx:
        sentences.append(coll.find_one({"_id": i})['sentence'])
    options = []
    keys = list(model.vocab.keys())
    nKeys = len(keys)
    for s in sentences:
        l = len(s)
        idx = list(range(0, nKeys))
        random.shuffle(idx)
        candidate = [keys[i] for i in idx[:3]]
#         count = 0
#         while count < 10:
        i = random.randint(0, l - 1)
#             if len(s[i]) > 4 or count == 9:
        options.append((i, [s[i]] + candidate))
#                 break
#             count += 1            
    return sentences, options

def cal_score(sentences, options):
    n = len(sentences)
    s = 0.
    ref = 0.
    for i in range(n):
        b = cloze_test(sentences[i], options[i][0], options[i][1])
        if b.argmax() == 0: s += 1.
        if random.randint(0, 3) == 0: ref += 1.
    return s/float(n), ref/float(n)


def cloze_test(string, index, candidates):
    temp = string[:]
    del temp[index]
    ref_vec = np.zeros(_VEC_SIZE_)
    l = 0
    for w in temp:
        if w in model.vocab:
            ref_vec += model[w]
        l += 1        
#     ref_vec /= float(l)
#     ref_vec = ref_vec / np.linalg.norm(ref_vec)
    nc = len(candidates)
    
    arr = np.zeros(nc)
    for i in range(nc):
        if candidates[i] in model.vocab:
            cand_vec = model[candidates[i]]
#             arr[i] = (ref_vec * cand_vec).sum()
            arr[i] = (ref_vec * (cand_vec / np.linalg.norm(cand_vec))).sum()
#     l, r = arr[:, j].min(), arr[:, j].max()
#     if r > l:
#         arr[:, j] = (arr[:, j] - l) / (r - l)
    return arr



def make_cloze_with_syntax(num):
    if num > 1000: num = 1000
        
    idx = []
    for i in range(num):
        idx.append(round(random.uniform(1, _COLL_NUM_)))
    idx = sorted(set(idx[: num]))
    
    SenSyn = []
    for i in idx:
        sen = coll.find_one({"_id": i})['sentence']
        syn = coll.find_one({"_id": i})['syntax_label']
        l = len(sen)
        if l == len(syn) and l > 5:
            SenSyn.append((sen, syn))
        
    Options = []
    keys = list(model.vocab.keys())
    nKeys = len(keys)
    for ss in SenSyn: # ss[0]:sen, ss[1]:syn
        l = len(ss[0])
        idx = list(range(0, nKeys))
        random.shuffle(idx)
        candidate = [keys[i] for i in idx[:3]]
#         count = 0
#         while count < 10:
        i = random.randint(0, l - 1)
#             if len(s[i]) > 4 or count == 9:
        Options.append((i, [ss[0][i]] + candidate))
#                 break
#             count += 1            
    return SenSyn, Options

def cloze_test2(string, syntax, index, cand_words):
    str_temp = string[:]
    syn_temp = syntax[:]
    del str_temp[index]
    del syn_temp[index]
    word_vec = np.zeros(_VEC_SIZE_)
    l = 0
    for w in str_temp:
        if w in model.vocab:
            word_vec += model[w]
        l += 1
    
    syn_vec = np.zeros(_SYN_VEC_SIZE_)
    l = 0
    for s in syn_temp:
        if s in model_syntax.vocab:
            syn_vec += model_syntax[s]
        l += 1  
    
    print(string)
    print(syntax)
    print(index)
    print(cand_words)
    print(mapW2S.map_w2s(cand_words))
#     ref_vec /= float(l)
#     ref_vec = ref_vec / np.linalg.norm(ref_vec)
    nc = len(cand_words)    
    word_score = np.zeros(nc)
    
    for i in range(nc):
        if cand_words[i] in model.vocab:
            cand_vec = model[cand_words[i]]
            word_score[i] = (word_vec * (cand_vec / np.linalg.norm(cand_vec))).sum()
    print(word_score)
    try:
        a = model_syntax[syntax[index + 1]]
        b = model_syntax[syntax[index - 1]]
        c = model_syntax[syntax[index + 2]]
        d = model_syntax[syntax[index - 2]]
        print('\n')
        s = model_syntax.most_similar(positive=[a + b + c + d])
        s = [x[0] for x in s]
        print(s)
    except:
        pass
    
    return ''

In [274]:
sen_syn, options = make_cloze_with_syntax(1)
for i in range(len(options)):
    cloze_test2(sen_syn[i][0], sen_syn[i][1], options[i][0], options[i][1])
    print('\n')
# for i in range(len(options)):
#     print(i+1)
#     print('{}\n{}\n{}\n--------------------------'.format(sen_syn[i][0], sen_syn[i][1], options[i]))

['在', '齋月', '頌', '讀', '、', '{{CD}}', '天', '正好', '讀', '{{CD}}', '卷', ',']
['P', 'Nd', 'VC', 'VC', 'PAUSECATEGORY', 'Neu', 'Nf', 'Da', 'VC', 'Neu', 'Nf', 'COMMACATEGORY']
9
['{{CD}}', '工業', '犯規', '古蹟']
{'工業': {'Na': 1365.0}, '犯規': {'Nv': 3.0, 'VA': 38.0}, '{{CD}}': {'A': 1.0, 'Nep': 2.0, 'Nc': 36.0, 'Cbb': 1816.0, 'VC': 4.0, 'Nb': 823.0, 'Neqa': 118.0, 'Nes': 5.0, 'VB': 4.0, 'PERIODCATEGORY': 1.0, 'Na': 76.0, 'Nd': 2625.0, 'VH': 62.0, 'D': 7990.0, 'Neu': 203846.0, 'Dfb': 13.0, 'FW': 401.0, 'Nf': 3.0, 'neu': 414.0, 'DM': 1.0, 'Di': 189.0}, '古蹟': {'Na': 415.0}}
[ 17.96525334  -8.91354979  -1.18152484   1.36981799]


['Nf', 'COMMACATEGORY', 'Da', 'PERIODCATEGORY', 'SEMICOLONCATEGORY', 'Na', 'P', 'VC', 'Ng', 'VG']




In [ ]:
for i in range(10):
    sentences, options = make_cloze(1000)
    score, ref = cal_score(sentences, options)
    print('{:.3f}, {:.3f}'.format(score, ref))

In [93]:
print(list(model_syntax.vocab.keys()))

['COMMACATEGORY', 'VC', 'EXCLAMATIONCATEGORY', 'VL', 'VAC', 'VI', 'Nh', 'D', 'DASHCATEGORY', 'VD', 'Dj', 'Di', 'Cba', 'Nep', 'COLONCATEGORY', 'SPCHANGECATEGORY', 'VF', 'Na', 'Da', 'VB', 'PERIODCATEGORY', 'Neqa', 'I', 'DM', 'DOTCATEGORY', 'Dk', 'Dfb', 'FW', 'VA', 'SEMICOLONCATEGORY', 'DE', 'SHI', 'EXCLANATIONCATEGORY', 'ETCCATEGORY', 'Nv', 'VHC', 'Neqb', 'P', 'b', 'Nd', 'Caa', 'Nf', 'Neu', 'VG', 'PARENTHESISCATEGORY', 'neu', 'Nes', 'A', 'VCL', 'VJ', 'Cab', 'Nc', 'VK', 'PAUSECATEGORY', 'V_2', 'QUESTIONCATEGORY', 'T', 'Ng', 'VE', 'VH', 'Nb', 'Dfa', 'Ncd', 'Cbb']


In [ ]:
class Word2Syn(object):
    def __init__(self, word_set, syn_set):
        import numpy as np
        import json
        self.word_set = word_set
        self.syn_set = syn_set

        lw = len(word_set)
        ls = len(syn_set)
        word_idx = {}
        for i in range(lw):
            word_idx[word_set[i]] = i
        with open('word_idx.json', 'w') as fp:
            json.dump(word_idx, fp)
        syn_idx = {}
        for i in range(ls):
            syn_idx[syn_set[i]] = i
        with open('syn_idx.json', 'w') as fp:
            json.dump(syn_idx, fp)
        self.word_idx = word_idx
        self.syn_idx = syn_idx
        self.word_syn_mapping = np.zeros((lw, ls))
    def generate(self, coll):
        import numpy as np
        for i in range(1, _COLL_NUM_ + 1):
            sen = coll.find_one({"_id": i})['sentence']
            syn = coll.find_one({"_id": i})['syntax_label']
            l = len(sen)
            if l == len(syn):
                for j in range(l):
                    try:
                        if (sen[j] in self.word_idx) and (syn[j] in self.syn_idx):
                            self.word_syn_mapping[self.word_idx[sen[j]], self.syn_idx[syn[j]]] += 1
                    except:
                        print(i)
                        print(j)
                        print(sen)
                        print(syn)
                        print('--------------------')
            else:
                print(i)
                print(sen)
                print(syn)
                print('******************')
        np.save('w2s_mapping', self.word_syn_mapping)

w2s = Word2Syn(list(model.vocab), list(model_syntax.vocab))
w2s.generate(coll)
        

In [28]:
i = 1059451
sen = coll.find_one({"_id": i})['sentence']
syn = coll.find_one({"_id": i})['syntax_label']
print(len(sen))
print(len(syn))


16
15


In [174]:
class MapW2S(object):
    def __init__(self):
        word_syn_mapping = np.load('w2s_mapping.npy')
        WORD_LEN, SYN_LEN = word_syn_mapping.shape
        with open('word_idx.json', 'r') as fp:
            word_idx = json.load(fp)
        with open('syn_idx.json', 'r') as fp:
            syn_idx = json.load(fp)
        word = ['abc' for i in range(WORD_LEN)]
        syn = ['abc' for i in range(SYN_LEN)]
        for w in word_idx:
            word[word_idx[w]] = w
        for s in syn_idx:
            syn[syn_idx[s]] = s
        self.word = word
        self.word_idx = word_idx
        self.syntax = syn
        self.syntax_idx = syn_idx
        self.mapping = word_syn_mapping
    
    def get_words(self, idx_list):
        return [self.word[i] for i in idx_list]
    def get_syntax(self, idx_list):
        return [self.syntax[i] for i in idx_list]

    def map_w2s(self, w_list):
        result = {}
        for w in w_list:
            syn = {}
            isyn = [x[0] for x in np.argwhere(self.mapping[self.word_idx[w], :] > 0)]
            for i in isyn:
                syn[self.syntax[i]] = self.mapping[self.word_idx[w], i]
            result[w] = syn
        return result

mapW2S = MapW2S()
l = mapW2S.map_w2s(['買', '了', '{{CD}}', '架', '當時', '算是', '先進', '稀罕', '的'])
print(l)


{'了': {'VK': 1.0, 'DE': 7.0, 'VJ': 51.0, 'VC': 14.0, 'VF': 1.0, 'T': 18533.0, 'VA': 1.0, 'Di': 62757.0}, '買': {'VC': 2419.0, 'VB': 1.0, 'Nv': 2.0}, '架': {'VC': 46.0, 'D': 3.0, 'Na': 49.0, 'Nf': 322.0, 'VA': 1.0}, '的': {'D': 1.0, 'Na': 6.0, 'T': 20697.0, 'Ng': 4.0, 'VE': 1.0, 'VC': 1.0, 'FW': 1.0, 'Nf': 1.0, 'DE': 561136.0, 'Ncd': 1.0, 'Di': 1.0}, '當時': {'Nd': 3554.0}, '先進': {'VH': 520.0, 'Na': 31.0}, '稀罕': {'VH': 11.0, 'VJ': 6.0}, '{{CD}}': {'A': 1.0, 'Nep': 2.0, 'Nc': 36.0, 'Cbb': 1816.0, 'VC': 4.0, 'Nb': 823.0, 'Neqa': 118.0, 'Nes': 5.0, 'VB': 4.0, 'PERIODCATEGORY': 1.0, 'Na': 76.0, 'Nd': 2625.0, 'VH': 62.0, 'D': 7990.0, 'Neu': 203846.0, 'Dfb': 13.0, 'FW': 401.0, 'Nf': 3.0, 'neu': 414.0, 'DM': 1.0, 'Di': 189.0}, '算是': {'VG': 761.0}}


In [155]:
a = model_syntax['PERIODCATEGORY']
a = a / a.sum()
b = model_syntax['Ng']
b = b / b.sum()
c = model_syntax['Nc']
c = c / c.sum()
d = model_syntax['DE']
d = d / d.sum()
print(c.sum())
print(d.sum())
model_syntax.most_similar([1 * (a + b + c + d)])

1.0
1.0


[('Nc', 0.8085162043571472),
 ('COMMACATEGORY', 0.8062970042228699),
 ('Caa', 0.7769955396652222),
 ('PERIODCATEGORY', 0.76839280128479),
 ('Na', 0.7463722229003906),
 ('A', 0.7328353524208069),
 ('SEMICOLONCATEGORY', 0.7314806580543518),
 ('DE', 0.7139890193939209),
 ('Ng', 0.663489043712616),
 ('Ncd', 0.6585845947265625)]

In [73]:
word_syn_mapping[0, 0]

0.0

775038
['全球', '最', '大', '療傷', '藥品', '供應商', '{{FW}}', '&', '{{FW}}', '目前', '正', '行銷', '{{CD}}', '種', '含有', '奈米', '大小', '結晶銀', '的', '抗菌', '藥膏', ',']
['Nc', 'Dfa', 'VH', 'VA', 'Na', 'Na', 'Nb', 'Nd', 'D', 'VCL', 'Neu', 'Nf', 'VJ', 'Na', 'Na', 'Na', 'DE', 'VH', 'Na', 'COMMACATEGORY']
******************
840658
['在', '{{FW}}', '{{FW}}', '溫暖', '的', '歌聲', '裡', ',']
['P', 'Nb', 'VHC', 'DE', 'Na', 'Ng', 'COMMACATEGORY']
******************
844325
['買', '了', '{{CD}}', '架', '當時', '算是', '先進', '稀罕', '的', '{{FW}}', '{{CD}}{{FW}}', '附', '{{CD}}', '㎜{{FW}}', '{{CD}}﹒{{CD}}', '鏡頭', '的', '相機', ',']
['VC', 'Di', 'Neu', 'Nf', 'Nd', 'VG', 'VH', 'VH', 'DE', 'Nb', 'VC', 'Neu', 'Nf', 'Neu', 'Na', 'DE', 'Na', 'COMMACATEGORY']
******************
1057158
['{{FW}}', '、', '{{FW}}', '、', '{{FW}}', '、', '{{FW}}', '、', '{{FW}}', '、', '{{FW}}', '{{FW}}', '都', '可以', '找到', '各', '種', '長短', '、', '合身度', '的', '性感', '下身', '。']
['Nb', 'PAUSECATEGORY', 'Nb', 'PAUSECATEGORY', 'Nb', 'PAUSECATEGORY', 'Nb', 'PAUSECATEGORY', 'Na', 'PAUSECATEGORY', 'Nb', 'D', 'D', 'VC', 'Nes', 'Nf', 'Na', 'PAUSECATEGORY', 'Na', 'DE', 'VH', 'Na', 'PERIODCATEGORY']
******************
1057368
['則', '新', '推', '{{FW}}', '{{FW}}', '系列', ',']
['D', 'D', 'VC', 'Nb', 'Na', 'COMMACATEGORY']
******************
1059451
['同', '{{CD}}', '時間', '姊妹台', '{{FW}}', '{{FW}}', '的', '「', '{{FW}}', '孩子王', '」', '則', '換', '了', '主持人', ',']
['Nes', 'Neu', 'Na', 'Nc', 'Nc', 'DE', 'PARENTHESISCATEGORY', 'FW', 'Na', 'PARENTHESISCATEGORY', 'D', 'VC', 'Di', 'Na', 'COMMACATEGORY']
******************
1093199
['顯示', '{{CD}}', '人', '的', '傳承', '關', '係', ',']
['VK', 'Neu', 'Na', 'DE', 'VC', 'Na', 'COMMACATEGORY']
******************
1195395
['那', '時候', '侯孝賢', '早', '就', '拍出', '「', '童年', '往事', '」', '、', '」', ',']
['Nep', 'Na', 'Nb', 'D', 'D', 'VC', 'PARENTHESISCATEGORY', 'Na', 'Na', 'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'Nb', 'PARENTHESISCATEGORY', 'COMMACATEGORY']
******************
1199895
['其', '股權', '結構', '是', '楊張美智', '(', '楊天生', '夫人', ')', '、', '郭政權', '(', '長{{CD}}', '實業', '代表', ')', '、', '王財旺', '(', '長{{CD}}', '實業', '代表', ')', '、', '簡肇涵', '(', '長{{CD}}', '實業', '代表', ')', '均', '分別', '持有', '泛亞', '股權', '{{CD}}.', '{{CD}}%', ',']
['Nep', 'Na', 'Na', 'SHI', 'Nb', 'PARENTHESISCATEGORY', 'Nb', 'Na', 'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'Nb', 'PARENTHESISCATEGORY', 'Nb', 'Na', 'Na', 'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'Nb', 'PARENTHESISCATEGORY', 'Nb', 'Na', 'Na', 'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'Nb', 'PARENTHESISCATEGORY', 'Nb', 'Na', 'Na', 'PARENTHESISCATEGORY', 'D', 'D', 'VJ', 'Nb', 'Na', 'Neqa', 'COMMACATEGORY']
******************